In [ ]:
import pandas as pd
import numpy as np

# assume around 2.5 mm of melting/°C per day -> 0.1mm/h/°C roughly

# start with a very simple model of snow the snow cover (only temperature and snow fall)

snowfall is in cm for the historic dataset

In [ ]:
histWeather = pd.read_csv("interpolHistWeather.csv")

In [ ]:
histWeather.columns

In [ ]:
#unique_pairs = list(set(zip(histWeather['latitude'], histWeather[ 'longitude'])))
#pair_index_dict = {pair: index for index, pair in enumerate(unique_pairs)}
#histWeather['weatherStation'] = list(map(pair_index_dict.get, zip(histWeather['latitude'], histWeather['longitude'])))
#unique_pairs

In [ ]:
histWeather['meltingInCM'] = -0.09 * histWeather['temperatureint'].clip(lower=0) #roughly 21mm/day/°
histWeather['snowHeightFlux'] = histWeather['snowfallint'] + histWeather['meltingInCM']

newDfList = []
for county in histWeather.County.unique():
    #test = histWeather.loc[(histWeather.latitude == pair[0]) & (histWeather.longitude==pair[1])]
    test = histWeather.loc[(histWeather.County == county)]

    snowcover = 0
    snowcoverArr = np.zeros((test.shape[0]))
    for i,snowFlux in enumerate(test['snowHeightFlux']):
        snowcover = max(snowcover + snowFlux,0)
        #print(snowcover,snowFlux,test['temperature'].iloc[i],test['snowfall'].iloc[i])
        snowcoverArr[i] = snowcover
    test['snowcover'] = snowcoverArr
    newDfList.append(test)
histWeather = pd.concat(newDfList)

In [ ]:
histWeather.to_csv('histWeatherSnowCover.csv')

In [ ]:
import plotly.express as px

histWeather['meltingInCM'] = -0.09 * histWeather['temperatureint'].clip(lower=0)
histWeather['snowHeightFlux'] = histWeather['snowfallint'] + histWeather['meltingInCM']
test = histWeather.loc[histWeather.County == 3]

snowcover = 0
snowcoverArr = np.zeros((test.shape[0]))
for i,snowFlux in enumerate(test['snowHeightFlux']):
    snowcover = max(snowcover + snowFlux,0)
    #print(snowcover,snowFlux,test['temperature'].iloc[i],test['snowfall'].iloc[i])
    snowcoverArr[i] = snowcover
test['snowcover'] = snowcoverArr

# Plot the 'Values' column over time using Plotly Express
fig = px.line(test, x='datetime', y='snowcover', title='Values over Time', labels={'Values': 'Values'})
fig.show()